# Workbook for Solving a TSP

In [64]:
import veroviz as vrv

In [65]:
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.0, the latest version available is 0.4.3. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [66]:
ORS_API_KEY='5b3ce3597851110001cf62480fb8f5604a0b485b911812eedd4d31cf'

# 1 -- Create some "nodes"

In [67]:
myBoundingRegion = [[43.01770145196991, -78.87840270996095], [42.878480017739044, -78.8756561279297], [42.83569550641454, -78.68270874023439], [42.96596996868038, -78.60717773437501], [43.04430359661548, -78.72528076171876]]
myBoundingRegion

[[43.01770145196991, -78.87840270996095],
 [42.878480017739044, -78.8756561279297],
 [42.83569550641454, -78.68270874023439],
 [42.96596996868038, -78.60717773437501],
 [43.04430359661548, -78.72528076171876]]

In [68]:
nodesDF = vrv.generateNodes(nodeType = 'customer',
                            nodeName = 'cust',
                            numNodes = 10,
                            incrementName = True,
                            nodeDistrib = 'uniformBB',
                            nodeDistribArgs = {'boundingRegion': myBoundingRegion},
                            snapToRoad = True,
                            dataProvider     = 'ORS-online',
                            dataProviderArgs = {'APIkey' : ORS_API_KEY})
nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters
0,1,42.945912,-78.730199,0,cust1,customer,1,glyphicon,info-sign,blue,1,pin,blue,1,None
1,2,42.902974,-78.720406,0,cust2,customer,2,glyphicon,info-sign,blue,2,pin,blue,2,None
2,3,42.892727,-78.744485,0,cust3,customer,3,glyphicon,info-sign,blue,3,pin,blue,3,None
3,4,42.920308,-78.726628,0,cust4,customer,4,glyphicon,info-sign,blue,4,pin,blue,4,None
4,5,42.940553,-78.772669,0,cust5,customer,5,glyphicon,info-sign,blue,5,pin,blue,5,None
5,6,42.957456,-78.685355,0,cust6,customer,6,glyphicon,info-sign,blue,6,pin,blue,6,None
6,7,42.985701,-78.680024,0,cust7,customer,7,glyphicon,info-sign,blue,7,pin,blue,7,None
7,8,42.891328,-78.755574,0,cust8,customer,8,glyphicon,info-sign,blue,8,pin,blue,8,None
8,9,42.972885,-78.667242,0,cust9,customer,9,glyphicon,info-sign,blue,9,pin,blue,9,None
9,10,42.939441,-78.722235,0,cust10,customer,10,glyphicon,info-sign,blue,10,pin,blue,10,None


In [69]:
len(nodesDF)

10

In [70]:
# View our nodes and bounding region:
vrv.createLeaflet(nodes = nodesDF, boundingRegion=myBoundingRegion)

In [71]:
# 2. Get time and distance matrices
[timeSec, distMeters] = vrv.getTimeDist2D(nodes = nodesDF,
                                          routeType = 'euclidean2D',
                                          speedMPS = vrv.convertSpeed(25, 'miles', 'hr', 'meters', 'second'))

In [72]:
#timeSec
timeSec[1,2]

432.7655377510959

In [73]:
# Optional, just to view the data in a table:
vrv.convertMatricesDictionaryToDataframe(timeSec)

,1,2,3,4,5,6,7,8,9,10
1,0.000000,432.765538,538.872006,255.841967,314.691703,346.966987,539.068540,573.370700,532.126663,86.718314
2,432.765538,0.000000,203.325912,178.199315,534.138959,599.030784,873.602953,281.884026,796.045893,362.734803
3,538.872006,203.325912,0.000000,303.627095,518.076542,774.958753,1037.179885,82.232298,976.306742,491.977545
4,255.841967,178.199315,303.627095,0.000000,391.910452,476.667924,733.705164,357.380014,679.117892,192.873763
5,314.691703,534.138959,518.076542,391.910452,0.000000,659.345271,811.699605,504.993639,834.144936,368.494986
6,346.966987,599.030784,774.958753,476.667924,659.345271,0.000000,283.446778,833.784003,202.500072,323.407382
7,539.068540,873.602953,1037.179885,733.705164,811.699605,283.446778,0.000000,1088.327914,157.900060,553.546147
8,573.370700,281.884026,82.232298,357.380014,504.993639,833.784003,1088.327914,0.000000,1036.093755,536.707332
9,532.126663,796.045893,976.306742,679.117892,834.144936,202.500072,157.900060,1036.093755,0.000000,521.282171
10,86.718314,362.734803,491.977545,192.873763,368.494986,323.407382,553.546147,536.707332,521.282171,0.000000


In [74]:
def tsp_cost(route, costDict):
    cost = 0
    
    i = route[0]
    for j in route[1:]:
        cost += costDict[i,j]
        i = j
        
    cost += costDict[i, route[0]]
    
    return cost

In [75]:
def tsp_neighbor(route):
    """
        This function computes a "subtour" solution to a TSP.
        
        Inputs
        ------
        route: list of Integer, indicating the node where the salesperson begins (and ends) the route
        
        Returns
        -------
        A random list of nodeIDs specifying another TSP route.
        """
    #import random
    from random import randint 
    
    #a = random.randint(0,len(route)-3)
    a = randint(0,len(route)-3)
    #b = random.randint(a+1, len(route)-2)
    b = randint(a+1, len(route)-2)
    
    newRoute = []
    newRoute.extend(route[0:a])
    
    subtour = route[a:b+1]
    subtour.reverse()
    newRoute.extend(subtour)
    
    newRoute.extend(route[b+1:len(route)-1])
    
    newRoute.append(newRoute[0])
    
    return newRoute

In [76]:
#Example of subtour
a=[1,5,8,9,2,4,3,7,6]
tsp_neighbor(a)

[1, 5, 2, 9, 8, 4, 3, 7, 1]

In [77]:
def solve_tsp_nn(startNode, costDict, nodesDF): 
    """
    This function computes a "nearest neighbor" solution to a TSP.
    
    Inputs
    ------
    startNode: Integer, indicating the node where the salesperson begins (and ends) the route
    
    costDict: VeRoViz time or distance dictionary.
    
    nodesDF: VeRoViz nodes dataframe
    
    Returns
    -------
    An ordered list of nodeIDs specifying a TSP route.
    """
    
    # Solve the TSP with a "nearest neighbor" heuristic
    nn_route = []

    # Start our route by visiting the startNode
    nn_route.append(startNode)

    # Initialize a list of unvisited nodes
    unvisitedNodes = list(nodesDF[nodesDF['id'] != startNode]['id'])

    # Let i represent our "current" location:
    i = startNode

    while len(unvisitedNodes) > 0:
        # Initialize minTime to a huge value
        minTime = float('inf')

        # Find the nearest unvisited node to our current node:
        for j in unvisitedNodes:
            if (costDict[i,j] < minTime):
                nextNode = j
                minTime = costDict[i,j]

        # Update our salesperson's location
        i = nextNode

        # Append nextNode to our route:
        nn_route.append(nextNode)

        # Remove nextNode from our list of unvisitedNodes:
        unvisitedNodes.remove(nextNode)

    nn_route.append(startNode)

    return nn_route    

In [78]:
def solveTSP_SA(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    T0=4000
    I=3
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initialize by calling your nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Simulated annealing (SA) procedure
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)    
    Tcur = T0
    print('Current Solution : Xcur =',Xcur,'Zcur =',Zcur)
    Xbest = Xcur
    Zbest = Zcur
    print('Best known Solution : Xbest =',Xbest,'Zbest =',Zbest)
    
    def solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime):
        print('Tcur=',Tcur)
        for i in range(1,I+1):
            #Generate a neighbor solution, Xcount
            # call tsp_neighbor(route)
            # ...
            # ...
            Xcount=tsp_neighbor(X0)
            Z = tsp_cost(Xcount,costDict)
            print('i=',i)
            print('Xcount=',Xcount,'Zcount=',Z)

            if (Z<Zcur):
                print('Z-Zcur=',Z-Zcur)
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Z)


            else:
                C=Z-Zcur
                print('C=',C)
                if(random()<=m.exp(-C/Tcur)):
                    Xcur = Xcount
                    Zcur = Z
                    print('Xcur=',Xcur,'Zcur=',Zcur)


            if(Z < Zbest):
                print('Z-Zbest=',Z-Zbest)
                Zbest = Z
                Xbest = Xcur
                print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        #print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
            
        runTime=time.time()-start_time
        print(runTime)
        if(Tcur<Tfinal or runTime>=timeLimit):
            if(Tcur<Tfinal):
                print('Tcur=',Tcur,'< Tfinal=',Tfinal)
                print('runTime =',runTime,',cutoffTime =',cutoffTime)
            elif(runTime>=cutoffTime):
                print('runTime =',runTime,'> cutoffTime =',cutoffTime)
                print('Tcur=',Tcur,',Tfinal=',Tfinal)
            return Xbest

        return solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    
    
    XBEST=solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    totalruntime=time.time()-start_time
    print('totaltime=',totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = XBEST,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [79]:
#%time
X_best=solveTSP_SA(nodesDF,timeSec,20)
X_best

Intial Solution : X0 = [1, 10, 4, 2, 3, 8, 5, 6, 9, 7, 1] Z0 = 2807.1571837844217
Current Solution : Xcur = [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zcur = 3166.114622322053
Best known Solution : Xbest = [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zbest = 3166.114622322053
Tcur= 4000
i= 1
Xcount= [1, 9, 6, 5, 8, 3, 2, 4, 10, 7, 1] Zcount= 3648.2116194595683
C= 482.09699713751525
Xcur= [1, 9, 6, 5, 8, 3, 2, 4, 10, 7, 1] Zcur= 3648.2116194595683
i= 2
Xcount= [1, 2, 4, 10, 3, 8, 5, 6, 9, 7, 1] Zcount= 3441.856040138979
Z-Zcur= -206.3555793205892
Xcur= [1, 2, 4, 10, 3, 8, 5, 6, 9, 7, 1] Zcur= 3441.856040138979
i= 3
Xcount= [1, 10, 3, 2, 4, 8, 5, 6, 9, 7, 1] Zcount= 3381.408682127964
Z-Zcur= -60.44735801101524
Xcur= [1, 10, 3, 2, 4, 8, 5, 6, 9, 7, 1] Zcur= 3381.408682127964
Xbest= [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zbest= 3166.114622322053
0.003548860549926758
Tcur= 3995
i= 1
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -613.9987170978184
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9,

Xcount= [1, 10, 4, 2, 3, 8, 6, 5, 9, 7, 1] Zcount= 3767.5924110548344
C= 386.1837289268706
Xcur= [1, 10, 4, 2, 3, 8, 6, 5, 9, 7, 1] Zcur= 3767.5924110548344
Xbest= [5, 8, 3, 2, 4, 10, 1, 6, 9, 7, 5] Zbest= 2767.409965030145
0.231550931930542
Tcur= 3840
i= 1
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
Z-Zcur= -660.0888450005314
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
i= 2
Xcount= [1, 10, 8, 3, 2, 4, 5, 6, 9, 7, 1] Zcount= 3037.907565687187
Z-Zcur= -69.59600036711618
Xcur= [1, 10, 8, 3, 2, 4, 5, 6, 9, 7, 1] Zcur= 3037.907565687187
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 69.59600036711618
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Xbest= [5, 8, 3, 2, 4, 10, 1, 6, 9, 7, 5] Zbest= 2767.409965030145
0.23287200927734375
Tcur= 3835
i= 1
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 0.0
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
i= 2
Xcount

i= 1
Xcount= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcount= 3015.4832040399056
C= 0.0
Xcur= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcur= 3015.4832040399056
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -248.07323900976007
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
C= 0.0
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.5106029510498047
Tcur= 3660
i= 1
Xcount= [1, 10, 4, 7, 9, 6, 5, 8, 3, 2, 1] Zcount= 3256.360031033888
C= 488.95006600374245
Xcur= [1, 10, 4, 7, 9, 6, 5, 8, 3, 2, 1] Zcur= 3256.360031033888
i= 2
Xcount= [1, 10, 4, 2, 3, 6, 5, 8, 9, 7, 1] Zcount= 4333.477321379184
C= 1077.1172903452962
Xcur= [1, 10, 4, 2, 3, 6, 5, 8, 9, 7, 1] Zcur= 4333.477321379184
i= 3
Xcount= [1, 4, 10, 2, 3, 8, 5, 6, 9, 7, 1] Zcount= 3160.816325939319
Z-Zcur= -1172.660995439865
Xcur= [1, 4, 10

Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.780937910079956
Tcur= 3485
i= 1
Xcount= [1, 10, 4, 7, 9, 6, 5, 8, 3, 2, 1] Zcount= 3256.360031033888
C= 375.198018850223
Xcur= [1, 10, 4, 7, 9, 6, 5, 8, 3, 2, 1] Zcur= 3256.360031033888
i= 2
Xcount= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcount= 3015.4832040399056
Z-Zcur= -240.87682699398238
Xcur= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcur= 3015.4832040399056
i= 3
Xcount= [1, 10, 4, 2, 3, 5, 8, 6, 9, 7, 1] Zcount= 3417.4401601543545
C= 401.956956114449
Xcur= [1, 10, 4, 2, 3, 5, 8, 6, 9, 7, 1] Zcur= 3417.4401601543545
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.7825679779052734
Tcur= 3480
i= 1
Xcount= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcount= 3015.4832040399056
Z-Zcur= -401.956956114449
Xcur= [1, 10, 4, 8, 3, 2, 5, 6, 9, 7, 1] Zcur= 3015.4832040399056
i= 2
Xcount= [2, 4, 10, 1, 3, 8, 5, 6, 9, 7, 2] Zcount= 3477.2376902193814
C= 461.75448617947586
Xcur= [2, 4, 10, 1, 3, 8, 5, 6, 9, 7, 2] Zcur= 3477.23

Z-Zcur= -596.075678035716
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -113.75204715351947
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.0410900115966797
Tcur= 3305
i= 1
Xcount= [1, 10, 3, 2, 4, 8, 5, 6, 9, 7, 1] Zcount= 3381.408682127964
C= 613.9987170978184
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
C= 0.0
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
i= 3
Xcount= [1, 10, 4, 2, 3, 7, 9, 6, 5, 8, 1] Zcount= 3796.406930683791
C= 1028.9969656536455
Xcur= [1, 10, 4, 2, 3, 7, 9, 6, 5, 8, 1] Zcur= 3796.406930683791
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.0429399013519287
Tcur= 3300
i= 1
Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
Z-Zcur= -915.244918500126
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Z

...

Xcur= [4, 10, 1, 2, 3, 8, 5, 6, 9, 7, 4] Zcur= 3256.3600310338884
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.276726007461548
Tcur= 2530
i= 1
Xcount= [1, 10, 4, 2, 3, 9, 6, 5, 8, 7, 1] Zcount= 4631.659481166766
C= 1375.299450132878
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
Z-Zcur= -375.19801885022343
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
i= 3
Xcount= [1, 4, 10, 2, 3, 8, 5, 6, 9, 7, 1] Zcount= 3160.816325939319
C= 279.65431375565413
Xcur= [1, 4, 10, 2, 3, 8, 5, 6, 9, 7, 1] Zcur= 3160.816325939319
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.27850079536438
Tcur= 2525
i= 1
Xcount= [1, 6, 5, 8, 3, 2, 4, 10, 9, 7, 1] Zcount= 3386.1879556343033
C= 225.37162969498422
Xcur= [1, 6, 5, 8, 3, 2, 4, 10, 9, 7, 1] Zcur= 3386.1879556343033
i= 2
Xcount= [1, 10, 4, 2, 9, 6, 5, 8, 3, 7, 1] Zcount= 4279.156988867999
C= 892.9690332336954
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 6, 5, 9, 7, 1] Zcount= 3767.5

Xcur= [1, 10, 8, 3, 2, 4, 5, 6, 9, 7, 1] Zcur= 3037.907565687187
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -270.4976006570414
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.5220699310302734
Tcur= 2350
i= 1
Xcount= [1, 8, 3, 2, 4, 10, 5, 6, 9, 7, 1] Zcount= 3157.310916903589
C= 389.9009518734433
Xcur= [1, 8, 3, 2, 4, 10, 5, 6, 9, 7, 1] Zcur= 3157.310916903589
i= 2
Xcount= [1, 10, 5, 8, 3, 2, 4, 6, 9, 7, 1] Zcount= 2800.101060245282
Z-Zcur= -357.20985665830676
Xcur= [1, 10, 5, 8, 3, 2, 4, 6, 9, 7, 1] Zcur= 2800.101060245282
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
C= 81.06095193838291
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.5368669033050537
Tcur= 2345
i= 1
Xcount= [1, 10, 4, 2, 5, 8, 3, 6, 9, 7, 1] Zcount= 3253.583713566513
C= 372.42170

i= 2
Xcount= [1, 10, 4, 2, 8, 3, 5, 6, 9, 7, 1] Zcount= 2898.7982010168125
Z-Zcur= -354.78551254970034
Xcur= [1, 10, 4, 2, 8, 3, 5, 6, 9, 7, 1] Zcur= 2898.7982010168125
i= 3
Xcount= [1, 10, 9, 6, 5, 8, 3, 2, 4, 7, 1] Zcount= 3711.3706963692457
C= 812.5724953524332
Xcur= [1, 10, 9, 6, 5, 8, 3, 2, 4, 7, 1] Zcur= 3711.3706963692457
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.79361891746521
Tcur= 2230
i= 1
Xcount= [1, 10, 7, 9, 6, 5, 8, 3, 2, 4, 1] Zcount= 2884.602994835183
Z-Zcur= -826.7677015340628
Xcur= [1, 10, 7, 9, 6, 5, 8, 3, 2, 4, 1] Zcur= 2884.602994835183
i= 2
Xcount= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zcount= 2699.192684542298
Z-Zcur= -185.4103102928848
Xcur= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zcur= 2699.192684542298
i= 3
Xcount= [1, 10, 4, 5, 8, 3, 2, 6, 9, 7, 1] Zcount= 2960.5538341754377
C= 261.3611496331396
Xcur= [1, 10, 4, 5, 8, 3, 2, 6, 9, 7, 1] Zcur= 2960.5538341754377
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
2.80462479591

Xcur= [3, 2, 4, 10, 1, 8, 5, 6, 9, 7, 3] Zcur= 3796.406930683791
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
3.026596784591675
Tcur= 2055
i= 1
Xcount= [1, 2, 4, 10, 3, 8, 5, 6, 9, 7, 1] Zcount= 3441.856040138979
Z-Zcur= -354.55089054481186
Xcur= [1, 2, 4, 10, 3, 8, 5, 6, 9, 7, 1] Zcur= 3441.856040138979
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -674.4460751088336
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
i= 3
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 340.0936010241576
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
3.0294320583343506
Tcur= 2050
i= 1
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -340.0936010241576
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zcount= 3166.114622322053
C= 39

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.945912,-78.730199,0,2.861307,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,2.861307,42.945992,-78.730360,0,5.189527,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,5.189527,42.946108,-78.730369,0,13.046307,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,13.046307,42.946498,-78.730314,0,15.901302,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,15.901302,42.946601,-78.730180,0,24.700000,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4338.200000,42.947033,-78.730073,0,4346.998698,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
948,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4346.998698,42.946601,-78.730180,0,4349.853693,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
949,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4349.853693,42.946498,-78.730314,0,4357.710473,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
950,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4357.710473,42.946108,-78.730369,0,4360.038693,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False


In [80]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)

In [81]:
def solveTSP_SA(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    T0=4000
    I=3
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initialize by calling your nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Simulated annealing (SA) procedure
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)    
    Tcur = T0
    print('Current Solution : Xcur =',Xcur,'Zcur =',Zcur)
    Xbest = Xcur
    Zbest = Zcur
    print('Best known Solution : Xbest =',Xbest,'Zbest =',Zbest)
    
    def solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime):
        print('Tcur=',Tcur)
        for i in range(1,I+1):
            #Generate a neighbor solution, Xcount
            # call tsp_neighbor(route)
            # ...
            # ...
            Xcount=tsp_neighbor(X0)
            Z = tsp_cost(Xcount,costDict)
            print('i=',i)
            print('Xcount=',Xcount,'Zcount=',Z)

            if (Z<Zcur):
                print('Z-Zcur=',Z-Zcur)
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Z)


            else:
                C=Z-Zcur
                print('C=',C)
                if(random()<=m.exp(-C/Tcur)):
                    Xcur = Xcount
                    Zcur = Z
                    print('Xcur=',Xcur,'Zcur=',Zcur)


            if(Z < Zbest):
                print('Z-Zbest=',Z-Zbest)
                Zbest = Z
                Xbest = Xcur
                print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        #print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
            
        runTime=time.time()-start_time
        print(runTime)
        if(Tcur<Tfinal or runTime>=timeLimit):
            if(Tcur<Tfinal):
                print('Tcur=',Tcur,'< Tfinal=',Tfinal)
                print('runTime =',runTime,',cutoffTime =',cutoffTime)
            elif(runTime>=cutoffTime):
                print('runTime =',runTime,'> cutoffTime =',cutoffTime)
                print('Tcur=',Tcur,',Tfinal=',Tfinal)
            return Xbest

        return solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    
    
    XBEST=solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    totalruntime=time.time()-start_time
    print('totaltime=',totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = XBEST,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [82]:
#%time
X_best=solveTSP_SA_loop(nodesDF,distMeters,2)
X_best

Intial Solution : X0 = [1, 10, 4, 2, 3, 8, 5, 6, 9, 7, 1] Z0 = 2807.1571837844217
Current Solution : Xcur = [1, 10, 4, 2, 3, 8, 6, 5, 9, 7, 1] Zcur = 3767.5924110548344
i= 1
Xcount= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zcount= 2699.192684542298
Z-Zcur= -1068.3997265125363
Xcur= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zcur= 2699.192684542298
Z-Zbest= -1068.3997265125363
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
Tcur= 3995
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.002836942672729492
i= 2
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 408.3108815120049
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 3990
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.003988981246948242
i= 3
Xcount= [1, 10, 4, 2, 3, 7, 9, 6, 5, 8, 1] Zcount= 3796.406930683791
C= 688.9033646294879
Xcur= [1, 10, 4, 2, 3, 7, 9, 6, 5, 8, 1] Zcur= 3796.406930683791
Tcur= 3985
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 

0.06395292282104492
i= 60
Xcount= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcount= 2767.4099650301455
Z-Zcur= -364.8267269616281
Xcur= [1, 10, 4, 2, 3, 8, 5, 7, 9, 6, 1] Zcur= 2767.4099650301455
Tcur= 3700
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.4716157913208008
i= 61
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 340.0936010241576
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 3695
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.47222399711608887
i= 62
Xcount= [1, 10, 5, 8, 3, 2, 4, 6, 9, 7, 1] Zcount= 2800.101060245282
Z-Zcur= -307.402505809021
Xcur= [1, 10, 5, 8, 3, 2, 4, 6, 9, 7, 1] Zcur= 2800.101060245282
Tcur= 3690
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.472719669342041
i= 63
Xcount= [1, 10, 4, 2, 5, 8, 3, 6, 9, 7, 1] Zcount= 3253.583713566513
C= 453.4826533212308
Xcur= [1, 10, 4, 2, 5, 8, 3, 6, 9, 7, 1] Zcur= 3253.583713566513
Tcur= 3685
Xbest= [1, 5, 8, 3, 2

Xcount= [1, 10, 6, 5, 8, 3, 2, 4, 9, 7, 1] Zcount= 3414.308622676661
Z-Zcur= -27.547417462318208
Xcur= [1, 10, 6, 5, 8, 3, 2, 4, 9, 7, 1] Zcur= 3414.308622676661
Tcur= 3435
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.9140279293060303
i= 114
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
Z-Zcur= -306.8050566223578
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 3430
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.9145007133483887
i= 115
Xcount= [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zcount= 3166.114622322053
C= 58.61105626774997
Xcur= [1, 10, 4, 2, 3, 8, 7, 9, 6, 5, 1] Zcur= 3166.114622322053
Tcur= 3425
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
0.9153118133544922
i= 116
Xcount= [1, 10, 4, 2, 3, 8, 9, 6, 5, 7, 1] Zcount= 3992.0568445947915
C= 825.9422222727385
Xcur= [1, 10, 4, 2, 3, 8, 9, 6, 5, 7, 1] Zcur= 3992.0568445947915
Tcur= 3420
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] 

Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.1144287586212158
i= 148
Xcount= [1, 10, 4, 2, 3, 5, 8, 6, 9, 7, 1] Zcount= 3417.4401601543545
Z-Zcur= -574.616684440437
Xcur= [1, 10, 4, 2, 3, 5, 8, 6, 9, 7, 1] Zcur= 3417.4401601543545
Tcur= 3260
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.1150047779083252
i= 149
Xcount= [1, 10, 4, 2, 3, 6, 5, 8, 9, 7, 1] Zcount= 4333.477321379184
C= 916.0371612248296
Tcur= 3255
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.1153500080108643
i= 150
Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
Z-Zcur= -536.2781479706896
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
Tcur= 3250
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.1163246631622314
i= 151
Xcount= [3, 2, 4, 10, 1, 8, 5, 6, 9, 7, 3] Zcount= 3796.406930683791
C= 915.244918500126
Tcur= 3245
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.1174447536468

C= 486.17374853636784
Xcur= [1, 10, 4, 2, 5, 8, 3, 6, 9, 7, 1] Zcur= 3253.583713566513
Tcur= 3035
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.398514747619629
i= 194
Xcount= [3, 2, 4, 10, 1, 8, 5, 6, 9, 7, 3] Zcount= 3796.406930683791
C= 542.8232171172781
Xcur= [3, 2, 4, 10, 1, 8, 5, 6, 9, 7, 3] Zcur= 3796.406930683791
Tcur= 3030
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.3997838497161865
i= 195
Xcount= [1, 6, 5, 8, 3, 2, 4, 10, 9, 7, 1] Zcount= 3386.1879556343033
Z-Zcur= -410.21897504948765
Xcur= [1, 6, 5, 8, 3, 2, 4, 10, 9, 7, 1] Zcur= 3386.1879556343033
Tcur= 3025
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.4001538753509521
i= 196
Xcount= [1, 10, 4, 2, 3, 8, 9, 6, 5, 7, 1] Zcount= 3992.0568445947915
C= 605.8688889604882
Xcur= [1, 10, 4, 2, 3, 8, 9, 6, 5, 7, 1] Zcur= 3992.0568445947915
Tcur= 3020
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.4009547233581543
i= 197
Xcount= [9, 6, 5, 8, 3

Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
Tcur= 2765
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.6814069747924805
i= 248
Xcount= [1, 10, 4, 2, 3, 6, 5, 8, 9, 7, 1] Zcount= 4333.477321379184
C= 1452.3153091955191
Tcur= 2760
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.6827828884124756
i= 249
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 226.3415538706381
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 2755
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.6842339038848877
i= 250
Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
Z-Zcur= -226.3415538706381
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
Tcur= 2750
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.6850807666778564
i= 251
Xcount= [1, 10, 5, 8, 3, 2, 4, 6, 9, 7, 1] Zcount= 2800.101060245282
Z-Zcur= -81.06095193838291
Xcur= [1, 10, 5,

Xcount= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcount= 2881.162012183665
Z-Zcur= -3.4409826515179702
Xcur= [1, 10, 4, 2, 3, 8, 5, 6, 7, 9, 1] Zcur= 2881.162012183665
Tcur= 2355
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.904865026473999
i= 330
Xcount= [1, 10, 4, 9, 6, 5, 8, 3, 2, 7, 1] Zcount= 4023.7786541863456
C= 1142.6166420026807
Xcur= [1, 10, 4, 9, 6, 5, 8, 3, 2, 7, 1] Zcur= 4023.7786541863456
Tcur= 2350
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.905472993850708
i= 331
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
Z-Zcur= -916.2750881320426
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 2345
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192684542298
1.9060699939727783
i= 332
Xcount= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcount= 3107.503566054303
C= 0.0
Xcur= [1, 10, 4, 2, 3, 8, 5, 9, 6, 7, 1] Zcur= 3107.503566054303
Tcur= 2340
Xbest= [1, 5, 8, 3, 2, 4, 10, 6, 9, 7, 1] Zbest= 2699.192

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.945912,-78.730199,0,2.861307,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,2.861307,42.945992,-78.730360,0,5.189527,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,5.189527,42.946108,-78.730369,0,13.046307,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,13.046307,42.946498,-78.730314,0,15.901302,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,15.901302,42.946601,-78.730180,0,24.700000,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4338.200000,42.947033,-78.730073,0,4346.998698,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
948,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4346.998698,42.946601,-78.730180,0,4349.853693,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
949,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4349.853693,42.946498,-78.730314,0,4357.710473,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False
950,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,4357.710473,42.946108,-78.730369,0,4360.038693,...,darkgray,None,211.0,211.0,-,No category,Asphalt,Street,0,False


In [83]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)